In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import tree
from sklearn.model_selection import train_test_split

In [2]:
ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is ACC0-1167

 Directory of C:\Users\shand\Documents\Siddy\ExcelR\Assignment\Random Forest

19-01-2022  01:10    <DIR>          .
19-01-2022  00:27    <DIR>          ..
16-01-2022  21:18    <DIR>          .ipynb_checkpoints
14-01-2022  03:24            17,023 Company_Data.csv
14-01-2022  03:24            21,837 Fraud_check.csv
14-01-2022  03:24             1,378 problem statement(company data).txt
14-01-2022  03:24               133 Problem_statement(Fraudcheck).txt
19-01-2022  01:10            54,610 Random Forest.ipynb
               5 File(s)         94,981 bytes
               3 Dir(s)  202,945,929,216 bytes free


## Company Data

In [3]:
company = pd.read_csv('Company_Data.csv')

In [4]:
company

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


In [5]:
company1 = company.copy()

In [6]:
company1.Sales.max(), '\n', company1.Sales.min()

(16.27, '\n', 0.0)

In [7]:
pd.cut(company1.Sales, bins = 3).value_counts(sort = False)

(-0.0163, 5.423]    102
(5.423, 10.847]     247
(10.847, 16.27]      51
Name: Sales, dtype: int64

In [8]:
labels = ['Low', 'Medium', 'High']
company1.Sales = pd.cut(company1.Sales, bins = 3, labels = labels) 

In [9]:
company1.Sales.value_counts()

Medium    247
Low       102
High       51
Name: Sales, dtype: int64

In [10]:
company1

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,Medium,138,73,11,276,120,Bad,42,17,Yes,Yes
1,High,111,48,16,260,83,Good,65,10,Yes,Yes
2,Medium,113,35,10,269,80,Medium,59,12,Yes,Yes
3,Medium,117,100,4,466,97,Medium,55,14,Yes,Yes
4,Low,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,High,138,108,17,203,128,Good,33,14,Yes,Yes
396,Medium,139,23,3,37,120,Medium,55,11,No,Yes
397,Medium,162,26,12,368,159,Medium,40,18,Yes,Yes
398,Medium,100,79,7,284,95,Bad,50,12,Yes,Yes


In [11]:
label_encoder = preprocessing.LabelEncoder()
company1.Sales = label_encoder.fit_transform(company1.Sales)

In [12]:
company1.Sales.value_counts()

2    247
1    102
0     51
Name: Sales, dtype: int64

In [13]:
company1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sales        400 non-null    int32 
 1   CompPrice    400 non-null    int64 
 2   Income       400 non-null    int64 
 3   Advertising  400 non-null    int64 
 4   Population   400 non-null    int64 
 5   Price        400 non-null    int64 
 6   ShelveLoc    400 non-null    object
 7   Age          400 non-null    int64 
 8   Education    400 non-null    int64 
 9   Urban        400 non-null    object
 10  US           400 non-null    object
dtypes: int32(1), int64(7), object(3)
memory usage: 32.9+ KB


In [14]:
company1.ShelveLoc = label_encoder.fit_transform(company1.ShelveLoc)
company1.Urban = label_encoder.fit_transform(company1.Urban)
company1.US = label_encoder.fit_transform(company1.US)

In [15]:
company1

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,2,138,73,11,276,120,0,42,17,1,1
1,0,111,48,16,260,83,1,65,10,1,1
2,2,113,35,10,269,80,2,59,12,1,1
3,2,117,100,4,466,97,2,55,14,1,1
4,1,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...,...
395,0,138,108,17,203,128,1,33,14,1,1
396,2,139,23,3,37,120,2,55,11,0,1
397,2,162,26,12,368,159,2,40,18,1,1
398,2,100,79,7,284,95,0,50,12,1,1


In [16]:
company1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Sales        400 non-null    int32
 1   CompPrice    400 non-null    int64
 2   Income       400 non-null    int64
 3   Advertising  400 non-null    int64
 4   Population   400 non-null    int64
 5   Price        400 non-null    int64
 6   ShelveLoc    400 non-null    int32
 7   Age          400 non-null    int64
 8   Education    400 non-null    int64
 9   Urban        400 non-null    int32
 10  US           400 non-null    int32
dtypes: int32(4), int64(7)
memory usage: 28.2 KB


In [18]:
x = company1.iloc[:,1:]
y = company1.iloc[:,0]

### Random Forest Classifier

In [22]:
num_trees = 100
max_features = 3
kfold = KFold(n_splits = 8)
model = RandomForestClassifier(n_estimators = num_trees, max_features = max_features, random_state = 10)
results = cross_val_score(model, x, y, cv = kfold)
print(results.mean())

0.7225


In [25]:
model.fit(x, y)

RandomForestClassifier(max_features=3, random_state=10)

In [26]:
model.feature_importances_

array([0.12448473, 0.10619595, 0.0745164 , 0.1157829 , 0.20147909,
       0.14634865, 0.13132484, 0.06294801, 0.01928143, 0.01763798])

## Fraud Check

In [28]:
fraud = pd.read_csv("Fraud_check.csv")
fraud

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [29]:
fraud1 = fraud.copy()

In [30]:
fraud1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


In [31]:
fraud1.Undergrad.value_counts()

YES    312
NO     288
Name: Undergrad, dtype: int64

In [32]:
fraud1['Marital.Status'].value_counts()

Single      217
Married     194
Divorced    189
Name: Marital.Status, dtype: int64

In [33]:
fraud1.Urban.value_counts()

YES    302
NO     298
Name: Urban, dtype: int64

In [34]:
fraud1.Undergrad = label_encoder.fit_transform(fraud1.Undergrad)
fraud1["Marital.Status"] = label_encoder.fit_transform(fraud1["Marital.Status"])
fraud1.Urban = label_encoder.fit_transform(fraud.Urban)

In [35]:
fraud1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Undergrad        600 non-null    int32
 1   Marital.Status   600 non-null    int32
 2   Taxable.Income   600 non-null    int64
 3   City.Population  600 non-null    int64
 4   Work.Experience  600 non-null    int64
 5   Urban            600 non-null    int32
dtypes: int32(3), int64(3)
memory usage: 21.2 KB


In [36]:
n = []
for i in fraud1["Taxable.Income"]:
    if i <= 30000:
        n.append('Risky')
    else:
        n.append('Good')

In [37]:
fraud1['Taxable.Income'] = n

In [38]:
fraud1['Taxable.Income'].value_counts()

Good     476
Risky    124
Name: Taxable.Income, dtype: int64

In [39]:
fraud1['Taxable.Income'] = label_encoder.fit_transform(fraud1['Taxable.Income'])
fraud1['Taxable.Income'].value_counts()

0    476
1    124
Name: Taxable.Income, dtype: int64

In [40]:
x = fraud1.loc[:, fraud1.columns != 'Taxable.Income']
y = fraud1.iloc[:,2] 

### Random Forest Classifier

In [41]:
num_trees = 100
max_features = 3
kfold = KFold(n_splits = 8)
model = RandomForestClassifier(n_estimators = num_trees, max_features = max_features, random_state = 10)
results = cross_val_score(model, x, y, cv = kfold)
print(results.mean())

0.745


In [42]:
model.fit(x, y)

RandomForestClassifier(max_features=3, random_state=10)

In [43]:
model.feature_importances_

array([0.04685838, 0.08605506, 0.54628594, 0.27016582, 0.0506348 ])